In [2]:
from langchain.llms import GooglePalm
from keys import google_api_key
llm = GooglePalm(google_api_key=google_api_key,temperature=0.2)
res=llm("how oceans produce more o2 than trees")
print(res)

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


The ocean produces more oxygen than trees because it is much larger and contains more phytoplankton, which are the main producers of oxygen in the ocean. Phytoplankton are microscopic plants that live in the upper layers of the ocean and use sunlight to photosynthesize, producing oxygen as a byproduct. Trees also produce oxygen through photosynthesis, but they are much smaller than the ocean and cover a much smaller area. In fact, the entire area of the ocean is about 145 million square kilometers, while the total area of forests is only about 4 billion hectares. This means that the ocean produces about 50 times more oxygen than forests.

In addition to being larger, the ocean also contains more phytoplankton than forests. Phytoplankton are found in all parts of the ocean, from the surface to the depths, while trees are only found on land. This means that the ocean has a much greater capacity to produce oxygen than forests.

The ocean is also a more efficient producer of oxygen than fo

In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db_user = "root"
db_password = "vivek123"
db_host = "localhost"
db_name = "k_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain=SQLDatabaseChain.from_llm(llm,db,verbose=True)


In [5]:
q1= db_chain.run("what are different brand t-shirts available")



> Entering new SQLDatabaseChain chain...
what are different brand t-shirts available
SQLQuery:

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


SELECT DISTINCT brand FROM t_shirts
SQLResult: [('Van Huesen',), ('Levi',), ('Nike',), ('Adidas',)]
Answer:Van Huesen, Levi, Nike, Adidas
> Finished chain.


In [6]:
q2=db_chain.run("display the total quantity of white Nike t-shirts available")



> Entering new SQLDatabaseChain chain...
display the total quantity of white Nike t-shirts available
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(30,), (57,)]
Answer:30
> Finished chain.


In [7]:
q3=db_chain.run("select sum(stock_quantity) where branc='Nike' and color='White';")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where branc='Nike' and color='White';
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White'
SQLResult: [(Decimal('87'),)]
Answer:87
> Finished chain.


In [8]:
db_chain.run("what are the different color t-shirts available in each brand")



> Entering new SQLDatabaseChain chain...
what are the different color t-shirts available in each brand
SQLQuery:SELECT brand, color FROM t_shirts GROUP BY brand, color ORDER BY brand
SQLResult: [('Van Huesen', 'Red'), ('Van Huesen', 'Blue'), ('Van Huesen', 'Black'), ('Van Huesen', 'White'), ('Levi', 'Red'), ('Levi', 'Blue'), ('Levi', 'Black'), ('Levi', 'White'), ('Nike', 'Red'), ('Nike', 'Blue'), ('Nike', 'Black'), ('Nike', 'White'), ('Adidas', 'Red'), ('Adidas', 'Blue'), ('Adidas', 'Black'), ('Adidas', 'White')]
Answer:Van Huesen: Red, Blue, Black, White
Levi: Red, Blue, Black, White
Nike: Red, Blue, Black, White
Adidas: Red, Blue, Black, White
> Finished chain.


'Van Huesen: Red, Blue, Black, White\nLevi: Red, Blue, Black, White\nNike: Red, Blue, Black, White\nAdidas: Red, Blue, Black, White'

In [9]:
q3=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='White';")
q5=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='Red';")
q6=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='Black';")
q7=db_chain.run("select sum(stock_quantity) where brand='Adidas' and color='Black';")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='White';
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White'
SQLResult: [(Decimal('87'),)]
Answer:87
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='Red';
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'Red'
SQLResult: [(Decimal('136'),)]
Answer:136
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='Black';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Black'
SQLResult: [(Decimal('179'),)]
Answer:179
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Adidas' and color='Black';
SQLQuery:SELECT SUM(stock_quantity)
FROM t_shirts
WHERE brand = 'Adidas'
AND color = 'Black'
SQLResult:

In [69]:
# llm will be confused when the question is not clear and give the wrong results
# for that reason we use few shot learning where question and answer are specified
# we need to save as a dictonary

few_shots = [
    {"Question": "How many white Nike t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='White' and brand='Nike';",
     'SQLResult':"Result of SQL query",
      'Answer': q3
     },
     {
     "Question": "How many Black Adidas t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas';",
     'SQLResult':"Result of SQL query",
      "Answer": q6   
     },
      {
     "Question": "what is the revenue generatd if all the Black nike t shirts with discount sold out",
     "SQLQuery": "select sum(t1.price*t1.stock_quantity-((t1.price*t1.stock_quantity*t2.pct_discount)/100)) as rev from t_shirts t1 join discounts t2 on t1.t_shirt_id=t2.t_shirt_id where t1.brand='Nike' and t1.color='Black';",
     'SQLResult':"Result of SQL query",
      "Answer": '799.200'   
     },
      {
     "Question": "what is the revenue generatd if all the Blue VanHuesen t shirts with discount sold out",
     "SQLQuery": "select sum(t1.price*t1.stock_quantity-((t1.price*t1.stock_quantity*t2.pct_discount)/100)) as rev from t_shirts t1 join discounts t2 on t1.t_shirt_id=t2.t_shirt_id where t1.brand='Van Huesen' and t1.color='Blue';",
     'SQLResult':"Result of SQL query",
      "Answer": '399.000'   
     },
      {
     "Question": "what is the revenue generatd if all the levi red t shirts without discount sold out",
     "SQLQuery": "select sum(price*stock_quantity) as rev from t_shirts where brand='Levi' and color='Red' and t_shirt_id not in(select t_shirt_id from discounts);",
     'SQLResult':"Result of SQL query",
      "Answer": '8886'   
     },
     {
     "Question": "what is the revenue generatd if all the white adidas t shirts without discount sold out",
     "SQLQuery": "select sum(price*stock_quantity) as rev from t_shirts where brand='Adidas' and color='White' and t_shirt_id not in(select t_shirt_id from discounts);",
     'SQLResult':"Result of SQL query",
      "Answer": '5024'   
     },
      {
     "Question": "what is the revenue generatd if all the van huesen blue t shirts without discount sold out",
     "SQLQuery": "select sum(price*stock_quantity) as rev from t_shirts where brand='Van Huesen' and color='Blue' and t_shirt_id not in(select t_shirt_id from discounts);",
     'SQLResult':"Result of SQL query",
      "Answer": '3872'   
     }
]


In [70]:
to_join=[" ".join(x.values())for x in few_shots]
to_join

["How many white Nike t-shirts are available select sum(stock_quantity) from t_shirts where color='White' and brand='Nike'; Result of SQL query 87",
 "How many Black Adidas t-shirts are available select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas'; Result of SQL query 179",
 "what is the revenue generatd if all the Black nike t shirts with discount sold out select sum(t1.price*t1.stock_quantity-((t1.price*t1.stock_quantity*t2.pct_discount)/100)) as rev from t_shirts t1 join discounts t2 on t1.t_shirt_id=t2.t_shirt_id where t1.brand='Nike' and t1.color='Black'; Result of SQL query 799.200",
 "what is the revenue generatd if all the Blue VanHuesen t shirts with discount sold out select sum(t1.price*t1.stock_quantity-((t1.price*t1.stock_quantity*t2.pct_discount)/100)) as rev from t_shirts t1 join discounts t2 on t1.t_shirt_id=t2.t_shirt_id where t1.brand='Van Huesen' and t1.color='Blue'; Result of SQL query 399.000",
 "what is the revenue generatd if all the le

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [71]:
from langchain.vectorstores import Chroma
vectorstore=Chroma.from_texts(to_join,embedding=embeddings,metadatas=few_shots)

In [72]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [74]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.prompts.prompt import PromptTemplate
query = input()


example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": query})


example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
new_chain(query)



> Entering new SQLDatabaseChain chain...
what is the revenue generatd if all the Black nike t shirts with discount sold out
SQLQuery:select sum(t1.price*t1.stock_quantity-((t1.price*t1.stock_quantity*t2.pct_discount)/100)) as rev from t_shirts t1 join discounts t2 on t1.t_shirt_id=t2.t_shirt_id where t1.brand='Nike' and t1.color='Black';
SQLResult: [(Decimal('799.200000'),)]
Answer:799.200
> Finished chain.


{'query': 'what is the revenue generatd if all the Black nike t shirts with discount sold out',
 'result': '799.200'}

In [40]:
"""
new_chain("How many Blue extras small Levi t-shirts are available")
new_chain("how many black Adidas t-shirts are available")
new_chain("select the total revenue that can be generated by selling nike")
new_chain("what is the costliest tshirt you have in your store and what is price of it along with its brand")
"""

'\nnew_chain("How many Blue extras small Levi t-shirts are available")\nnew_chain("how many black Adidas t-shirts are available")\nnew_chain("select the total revenue that can be generated by selling nike")\nnew_chain("what is the costliest tshirt you have in your store and what is price of it along with its brand")\n'

In [75]:
inp = input()
new_chain(inp)



> Entering new SQLDatabaseChain chain...
which brand has more number of xl tshirts
SQLQuery:select brand, count(*) as num_xl_tshirts from t_shirts where size='XL' group by brand order by num_xl_tshirts desc limit 1;
SQLResult: [('Adidas', 4)]
Answer:Adidas
> Finished chain.


{'query': 'which brand has more number of xl tshirts', 'result': 'Adidas'}